In [1]:
import json

with open('./osconfeed.json') as fp:
    feed = json.load(fp)
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [2]:
for key, value in sorted(feed['Schedule'].items()):
    print(f'{len(value):3} {key}')

  1 conferences
484 events
357 speakers
 53 venues


In [3]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [4]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [6]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [7]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

In [67]:
from collections import abc

# So we can use . notation accessing objects in JSON
class JavascriptJSON:
    def __init__(self, mapping):
        self.__data = dict(mapping)

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        try:
            return getattr(self.__data[name], name)
        except AttributeError:
            return JavascriptJSON.build(self.__data[name])

    def __dir__(self):
        return self.__data.key()

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [68]:
import json
raw_feed = json.load(open('./osconfeed.json'))
feed = JavascriptJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [55]:
print(type(feed))
#feed.__dir__()

<class '__main__.JavascriptJSON'>


In [56]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [57]:
for key, value in sorted(feed.Schedule.items()):
    print(f'{len(value):3} {key}')

  1 conferences
484 events
357 speakers
 53 venues


In [58]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [59]:
talk = feed.Schedule.events[40]
type(talk)

__main__.JavascriptJSON

In [60]:
talk.name

'There *Will* Be Bugs'

In [61]:
talk.speakers

[3471, 5199]

In [62]:
talk.flavor # Keyerror

KeyError: 'flavor'

In [69]:
# To prevent invalid attribute access, append an _ after the keyword:
# student.class will produce an error so we need to access like this student.class_
import keyword

def __init__(self, mapping):
    self.__data = {}
    for key, value in mapping.items():
        if keyword.iskeyword(key):
            key += '_'
        self.__data[key] = value

In [70]:
# using new
class JavascriptJSON:
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        try:
            return getattr(self.__data, name)
        except AttributeError:
            return JavascriptJSON(self.__data[name])

    def __dir__(self):
        return self.__data.keys()

In [71]:
# Goal
# records = load(JSON_PATH)
# speaker = records['speaker.3471']
import json

JSON_PATH = './osconfeed.json'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'

def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1] # to remove the s in the last character, also .removesuffix('s') works
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = Record(**raw_record)
    return records

SyntaxError: unterminated string literal (detected at line 3) (4288273551.py, line 3)

In [72]:
# Goal
# event = Record.fetch('event.33950')
# event
# <Event 'There *Will* Be Bugs'>
# event.venue
# <Record serial=1449>
# event.venue.name
import inspect
import json

JSON_PATH = './osconfeed.json'

class Record:
    __index = None
    
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'

    @staticmethod
    def fetch(key):
        if Record.__index is None:
            Record.__index = load()
        return Record.__index[key]

In [79]:
from functools import cached_property

class Event(Record):
    def __init__(self, **kwargs):
        self.__speaker_objs = None
        super().__init__(**kwargs)
    
    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        # if an event record had a key named 'fetch', then within that specific
        # Event instance, the reference self.fetch would retrieve the value of that field,
        # instead of the fetch class method that Event inherits from Record, hence:
        # self.__class__.fetch(key) not self.fetch(key)
        return self.__class__.fetch(key)

    @cached_property
    def speakers(self):
        spkr_serials = self.__dict__['speakers']
        fetch = self.__class__.fetch
        return [fetch(f'speaker.{key}')
                for key in spkr_serials]

    # @property
    # def speakers(self):
    #     if self.__speaker_objs is None: # cache
    #         spkr_serials = self.__dict__['speakers'] # recursion is avoided
    #         fetch = self.__class__.fetch
    #         self.__speaker_obj = [fetch(f'speaker.{key}')
    #                               for key in spkr_serials]
    #     return self.__speaker_obj

In [74]:
def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, Record)
        if inspect.isclass(cls) and issubclass(cls, Record):
            factory = cls
        else:
            factory = Record
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = factory(**raw_record)
    return records

In [85]:
# getter and setter for values that could be overwritten with `-`
# we prevent that here
def quantity(storage_name):
    def qty_getter(instance): # not self but in this case LineItem or instance (to be general)
        return instance.__dict__[storage_name]

    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    # @property
    # def weight(self):
    #     return self.__weight

    # @weight.setter
    # def weight(self, value):
    #     if value > 0:
    #         self.__weight = value
    #     else:
    #         raise ValueError('value must be > 0')

In [87]:
turkey = LineItem('thanksgiving turkey', 20, 35.23)
turkey.weight, turkey.price

(20, 35.23)

In [102]:
# the weight property overrides the weight instance attribute so 
# that every reference to self.weight or turkey.weight is handled
# by the property functions, and the only way to bypass the
# property logic is to access the instance __dict__ directly
turkey.__dict__

{'description': 'thanksgiving turkey', 'weight': 20, 'price': 35.23}

In [103]:
vars(turkey) # locals()

{'description': 'thanksgiving turkey', 'weight': 20, 'price': 35.23}

In [91]:
class Demo:
    pass

d = Demo()
d.color = 'red'
d.color

'red'

In [92]:
del d.color

In [93]:
d.color

AttributeError: 'Demo' object has no attribute 'color'

In [96]:
class BlackKnight:
    def __init__(self):
        self.phrases = [
            ('an arm', "'Tis but a scratch"),
            ('another arm', "It's just a flesh wound."),
            ('a leg', "I'm invincible!"),
            ('another leg', "All right, we'll call it a draw.")
        ]

    @property
    def member(self):
        print('next member is:')
        return self.phrases[0][0]

    # member = property(member_getter, fdel=member_deleter)
    @member.deleter
    def member(self):
        member, text = self.phrases.pop(0)
        print(f'BLACK KNIGHT (loses {member}) -- {text}')

In [97]:
knight = BlackKnight()
knight.member

next member is:


'an arm'

In [98]:
del knight.member

BLACK KNIGHT (loses an arm) -- 'Tis but a scratch


In [99]:
del knight.member

BLACK KNIGHT (loses another arm) -- It's just a flesh wound.


In [100]:
del knight.member

BLACK KNIGHT (loses a leg) -- I'm invincible!


In [101]:
del knight.member

BLACK KNIGHT (loses another leg) -- All right, we'll call it a draw.


In [104]:
"""
obj.__class__ = type(obj)

__getattr__ is only invoked after __getattribute__, and
only when __getattribute__ raises AttributeError.

To retrieve attributes of
the instance obj without triggering an infinite recursion, implementations of
__getattribute__ should use super().__getattribute__(obj, name).
"""

'\nobj.__class__ = type(obj)\n\n__getattr__ is only invoked after __getattribute__, and\nonly when __getattribute__ raises AttributeError.\n\nTo retrieve attributes of\nthe instance obj without triggering an infinite recursion, implementations of\n__getattribute__ should use super().__getattribute__(obj, name).\n'